In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from google.colab import files
from google.colab import drive
import pathlib

In [ ]:
import random

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = "1"

In [ ]:
def upload_drive():
  drive.mount('/content/drive')

upload_drive()

In [ ]:
!cp '/content/drive/My Drive/AARD/idb1.zip' .

In [ ]:
!unzip idb1.zip
!rm -rf __MACOSX/
!mkdir -p idb1small/ds/neg
!mkdir -p idb1small/ds/pos

In [ ]:
TARGET_RATIO = 4/3
TARGET_WIDTH = 1200
TARGET_HEIGHT = int(TARGET_WIDTH/TARGET_RATIO)

def resize_image(img, tw=TARGET_WIDTH, th=TARGET_HEIGHT): 
    w, h = img.size[0], img.size[1] 
    dw, dh = 0, 0 
    tr = tw / th 
    if w / h > tr: 
        dw = w - h * tr 
    elif w / h < tr: 
        dh = h - w / tr 
    if dw > 0 or dh > 0: 
        img = img.crop((dw//2, dh // 2, w - dw // 2, h - dh // 2))

    return img.resize((tw, th))

def resize_all_images(from_dir: str, to_dir: str, width: int, height: int): 
    """Resize all images from one directory and put result in another. 
       neg and pos should be the child directories of input dir. 
    """
    src_dir = pathlib.Path(from_dir)
    tgt_dir = pathlib.Path(to_dir) 
    for src_img_path in src_dir.glob('*/*.jpg'): 
        print(f"Resize image: {src_img_path}") 
        img = tf.keras.utils.load_img(src_img_path) 
        resized_img = resize_image(img, tw=width, th=height) 
        tgt_img_path = (str(src_img_path)).replace(from_dir, to_dir) 
        resized_img.save(tgt_img_path)
        print(f"Image {src_img_path} is resized as {tgt_img_path} to size {width}/{height}")

In [ ]:
resize_all_images('idb1/ds', 'idb1small/ds', TARGET_WIDTH, TARGET_HEIGHT)

In [ ]:
!mkdir -p test_data/neg
!mkdir -p test_data/pos

test_dir = pathlib.Path('test_data')

#Load model from drive

In [ ]:
# upload the model
model = tf.keras.models.load_model('/content/drive/My Drive/AARD/Saved_Models/all_model1')

# Check its architecture
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 900, 1200, 3)      0         
                                                                 
 rescaling_5 (Rescaling)     (None, 900, 1200, 3)      0         
                                                                 
 conv2d_12 (Conv2D)          (None, 900, 1200, 16)     448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 450, 600, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 450, 600, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 225, 300, 32)     0         
 g2D)                                                 

#Create the test dataset

In [ ]:
TARGET_RATIO = 4/3
TARGET_WIDTH = 1200
TARGET_HEIGHT = int(TARGET_WIDTH/TARGET_RATIO)

batch_size = 32
img_height = TARGET_HEIGHT
img_width = TARGET_WIDTH

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  # validation_split=0.0002,
  # subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
for img_data, img_label in test_ds.take(1000):
    model.evaluate(img_data, img_label)

#Test data

In [ ]:
!mkdir -p aug_test_data/neg
!mkdir -p aug_test_data/pos

import math
import pathlib
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
from skimage import transform

def augment_image(src_dir: str, tgt_dir: str):
    for img_file in pathlib.Path(src_dir).glob('*/*.jpg'):
        img = load_img(img_file)
        img_arr = img_to_array(img)
        angles = list(range(-180, 180, 45))
        for i, r in enumerate(angles):
            # rotate
            aug_image = transform.rotate(img_arr, angle=r, clip=False, mode='reflect')
            # save
            rotated_img_file = str(img_file).replace(src_dir, tgt_dir)
            path_elems = list(rotated_img_file.split('/'))
            path_elems[-1] = f"r{r}_" + path_elems[-1]
            img_tgt_file = '/'.join(path_elems)
            print(f"Generate aug image: {img_tgt_file}")
            save_img(img_tgt_file, aug_image)

In [ ]:
augment_image('test_data', 'aug_test_data')

In [ ]:
!cp aug_test_data/pos/r-180_Im062_1.jpg aug_test_data/neg/r-180_Im062_1.jpg

In [ ]:
aug_test_ds = tf.keras.utils.image_dataset_from_directory(
  'aug_test_data',
  # validation_split=0.0002,
  # subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=200)

Found 129 files belonging to 2 classes.


In [ ]:
seed_everything(42)

for img_data, img_label in aug_test_ds.take(10):
    model.evaluate(img_data, img_label)

5/5 [==============================] - 38s 7s/step - loss: 0.3810 - accuracy: 0.9922
